# Hyperparameter Optimization (Pain Recognition)

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import tensorflow as tf

from sklearn.model_selection import KFold
from tensorflow.keras.layers import Input, LSTM, Dense, Flatten, TimeDistributed, Conv1D, BatchNormalization, MaxPooling1D, Bidirectional, Concatenate, Dropout
from tensorflow.keras import Model
from keras.layers.advanced_activations import PReLU

import sys
module_path = os.path.abspath(os.getcwd() + '\\..')
if module_path not in sys.path:
    sys.path.append(module_path)
from src.data.load_dataset import load_dataset
from src.lib.time_series_augmentation.utils.augmentation import jitter, rotation

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=10,
    restore_best_weights=True,
)

kf = KFold(n_splits=3, shuffle=True, random_state=42)
epochs = 50

In [ ]:
units = [64, 128, 256, 512]
activations = ['relu', 'tanh', 'sigmoid']
kernel_size = [3, 5]
learning_rate = [0.001, 0.0001, 0.00001]
optimizer = ['Adam', 'RMSprop', 'Nadam']
dropout = [0.0, 0.1, 0.2, 0.3]

## 1. Unimodal Body Modality (Skeleton)

In [ ]:
X_train, X_test, y_train, y_test = load_dataset('skeleton', binary=True)
X = X_train.copy()
y = y_train.copy()

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

n_length, n_features, n_outputs = X_train.shape[2], X_train.shape[-1], y_train.shape[1]

In [ ]:
jitter_X = jitter(X, sigma=0.03)
reshaped_X = X.reshape((X.shape[0], X.shape[2], X.shape[-1]))
rotated_X = rotation(reshaped_X)
rotated_X = rotated_X.reshape((rotated_X.shape[0], 1, rotated_X.shape[1], rotated_X.shape[-1]))
augmented_X = np.concatenate((rotated_X[0:200], jitter_X[0:200]), axis=0)
add_y = np.concatenate((y[0:200], y[0:200]), axis=0)
full_X = np.concatenate((augmented_X, X), axis=0)
full_y = np.concatenate((add_y, y), axis=0)
full_X.shape

### CNN + BiLSTM

In [ ]:
class CNNLSTM():
    def __init__(self, n_features, n_length, n_outputs, units, activation, kernel_size, learning_rate, optimizer, dropout):
        if optimizer == 'Adam':
            ops = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        elif optimizer == 'RMSprop':
            ops = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
        elif optimizer == 'Nadam':
            ops = tf.keras.optimizers.Nadam(learning_rate=learning_rate)

        input = Input(shape=(1, n_length, n_features))
        conv1d_1 = Conv1D(filters=units, kernel_size=kernel_size,
                          activation=activation)(input)
        bn_1 = BatchNormalization()(conv1d_1)
        conv1d_2 = Conv1D(filters=units, kernel_size=kernel_size,
                          activation=activation)(bn_1)
        maxpool_1 = TimeDistributed(MaxPooling1D(
            pool_size=2, strides=2, data_format='channels_first'))(conv1d_2)
        lstm_1 = TimeDistributed(Bidirectional(
            LSTM(units=units, return_sequences=True)))(maxpool_1)
        lstm_2 = TimeDistributed(Bidirectional(LSTM(units=units)))(lstm_1)
        flatten = Flatten()(lstm_2)
        dense_1 = Dense(units, activation=activation)(flatten)
        dropout_1 = Dropout(dropout)(dense_1)
        dense_2 = Dense(units, activation=activation)(dropout_1)
        output = Dense(units=n_outputs, activation='sigmoid')(dense_2)
        model = Model(inputs=input, outputs=output)
        model.compile(loss='binary_crossentropy', optimizer=ops, metrics=['accuracy'])
        self.model = model

    def train(self, X_train, y_train, X_val, y_val, epochs):
        history = self.model.fit(X_train, y_train, validation_data=(
            X_val, y_val), epochs=epochs, batch_size=32, callbacks=[early_stopping], verbose=2)
        return history

In [ ]:
best_acc = 0.0
body_cnnlstm_values = { 'units': 0, 'activation': '', 'kernel_size': 0, 'learning_rate' : 0.0, 'optimizer': '', 'dropout': 0.0 }

In [ ]:
for u in units:
    for a in activations:
        for k in kernel_size:
            for l in learning_rate:
                for o in optimizer:
                    for d in dropout:
                        print('\nUnits:', u, 'Activation:', a, 'Kernel size:', k, 'Learning rate:', l, 'Optimizer:', o, 'Dropout:', d)
                        for train_index, val_index in kf.split(full_X):
                            X_train, X_val = full_X[train_index], full_X[val_index]
                            y_train, y_val = full_y[train_index], full_y[val_index]
                            model = CNNLSTM(n_features, n_length, n_outputs, u, a, k, l, o, d)
                            history = model.train(X_train, y_train, X_val, y_val, epochs=epochs)
                            if (history.history['val_accuracy'][-1] > best_acc):
                                best_acc = history.history['val_accuracy'][-1]
                                body_cnnlstm_values = { 'units': u, 'activation': a, 'kernel_size': k, 'learning_rate' : l, 'optimizer': o, 'dropout': d }

In [ ]:
print(best_acc)
print(body_cnnlstm_values)

### RCNN

In [ ]:
class RCNN():
    def __init__(self, n_features, n_length, n_outputs, units, kernel_size, learning_rate, optimizer, dropout):
        if optimizer == 'Adam':
            ops = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        elif optimizer == 'RMSprop':
            ops = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
        elif optimizer == 'Nadam':
            ops = tf.keras.optimizers.Nadam(learning_rate=learning_rate)

        input = Input(shape=(1, n_length, n_features))
        conv1 = Conv1D(filters=units, kernel_size=kernel_size, padding='same')
        stack1 = conv1(input)
        stack2 = BatchNormalization()(stack1)
        stack3 = PReLU()(stack2)
        conv2 = Conv1D(filters=units, kernel_size=kernel_size,
                       padding='same', kernel_initializer='he_normal')
        stack4 = conv2(stack3)
        stack5 = Concatenate()([stack1, stack4])
        stack6 = BatchNormalization()(stack5)
        stack7 = PReLU()(stack6)
        conv3 = Conv1D(filters=units, kernel_size=kernel_size,
                       padding='same')
        stack8 = conv3(stack7)
        stack9 = Concatenate()([stack1, stack8])
        stack10 = BatchNormalization()(stack9)
        stack11 = PReLU()(stack10)
        stack12 = TimeDistributed(MaxPooling1D(
            (2), strides=2, data_format='channels_first'))(stack11)
        stack13 = Dropout(dropout)(stack12)
        flatten = Flatten()(stack13)
        output = Dense(units=n_outputs, activation='sigmoid')(flatten)
        model = Model(inputs=input, outputs=output)
        model.compile(loss='binary_crossentropy', optimizer=ops, metrics=['accuracy'])
        self.model = model

    def train(self, X_train, y_train, X_val, y_val, epochs, batch_size):
        history = self.model.fit(X_train, y_train, validation_data=(
            X_val, y_val), epochs=epochs, batch_size=batch_size, callbacks=[self.early_stopping], verbose=2)
        return history


In [ ]:
best_acc = 0.0
body_rcnn_values = { 'units': 0, 'kernel_size': 0, 'learning_rate' : 0.0, 'optimizer': '', 'dropout': 0.0 }

In [ ]:
for u in units:
    for k in kernel_size:
        for l in learning_rate:
            for o in optimizer:
                for d in dropout:
                    print('\nUnits:', u, 'Kernel size:', k, 'Learning rate:', l, 'Optimizer:', o, 'Dropout:', d)
                    for train_index, val_index in kf.split(full_X):
                        X_train, X_val = full_X[train_index], full_X[val_index]
                        y_train, y_val = full_y[train_index], full_y[val_index]
                        model = RCNN(n_features, n_length, n_outputs, u, k, l, o, d)
                        history = model.train(X_train, y_train, X_val, y_val, epochs=epochs)
                        if (history.history['val_accuracy'][-1] > best_acc):
                            best_acc = history.history['val_accuracy'][-1]
                            body_rcnn_values = { 'units': u, 'kernel_size': k, 'learning_rate' : l, 'optimizer': o, 'dropout': d }

In [ ]:
print(best_acc)
print(body_rcnn_values)

## 2. Unimodal Face modality (Action units + Head pose)

In [ ]:
X_train, X_test, y_train, y_test = load_dataset('AUs', binary=True)
X = X_train.copy()
y = y_train.copy()

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

n_length, n_features, n_outputs = X_train.shape[2], X_train.shape[-1], y_train.shape[1]

In [ ]:
jitter_X = jitter(X, sigma=0.03)
reshaped_X = X.reshape((X.shape[0], X.shape[2], X.shape[-1]))
rotated_X = rotation(reshaped_X)
rotated_X = rotated_X.reshape((rotated_X.shape[0], 1, rotated_X.shape[1], rotated_X.shape[-1]))
augmented_X = np.concatenate((rotated_X[0:200], jitter_X[0:200]), axis=0)
add_y = np.concatenate((y[0:200], y[0:200]), axis=0)
full_X = np.concatenate((augmented_X, X), axis=0)
full_y = np.concatenate((add_y, y), axis=0)
full_X.shape

### CNN + BiLSTM

In [ ]:
best_acc = 0.0
face_cnnlstm_values = { 'units': 0, 'activation': '', 'kernel_size': 0, 'learning_rate' : 0.0, 'optimizer': '', 'dropout': 0.0 }

In [ ]:
for u in units:
    for a in activations:
        for k in kernel_size:
            for l in learning_rate:
                for o in optimizer:
                    for d in dropout:
                        print('\nUnits:', u, 'Activation:', a, 'Kernel size:', k, 'Learning rate:', l, 'Optimizer:', o, 'Dropout:', d)
                        for train_index, val_index in kf.split(full_X):
                            X_train, X_val = full_X[train_index], full_X[val_index]
                            y_train, y_val = full_y[train_index], full_y[val_index]
                            model = CNNLSTM(n_features, n_length, n_outputs, u, a, k, l, o, d)
                            history = model.train(X_train, y_train, X_val, y_val, epochs=epochs)
                            if (history.history['val_accuracy'][-1] > best_acc):
                                best_acc = history.history['val_accuracy'][-1]
                                face_cnnlstm_values = { 'units': u, 'activation': a, 'kernel_size': k, 'learning_rate' : l, 'optimizer': o, 'dropout': d }

In [ ]:
print(best_acc)
print(face_cnnlstm_values)

### RCNN

In [ ]:
best_acc = 0.0
face_rcnn_values = { 'units': 0, 'kernel_size': 0, 'learning_rate' : 0.0, 'optimizer': '', 'dropout': 0.0 }

In [ ]:
for u in units:
    for k in kernel_size:
        for l in learning_rate:
            for o in optimizer:
                for d in dropout:
                    print('\nUnits:', u, 'Kernel size:', k, 'Learning rate:', l, 'Optimizer:', o, 'Dropout:', d)
                    for train_index, val_index in kf.split(full_X):
                        X_train, X_val = full_X[train_index], full_X[val_index]
                        y_train, y_val = full_y[train_index], full_y[val_index]
                        model = RCNN(n_features, n_length, n_outputs, u, k, l, o, d)
                        history = model.train(X_train, y_train, X_val, y_val, epochs=epochs)
                        if (history.history['val_accuracy'][-1] > best_acc):
                            best_acc = history.history['val_accuracy'][-1]
                            face_rcnn_values = { 'units': u, 'kernel_size': k, 'learning_rate' : l, 'optimizer': o, 'dropout': d }

In [ ]:
print(best_acc)
print(face_rcnn_values)